In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
%matplotlib inline

In [8]:
train_data = pd.read_csv('FINAL_TRAIN.csv', header=0)
train_data.head()

,spouse_age,typical_weekly_hours_worked,work_days_missed,family_income,age,asthma_still_2.0,asthma_still_10.0,overnight_hospitalization_12m_2,overnight_hospitalization_12m_10,diff_pay_med_bills_12m_2,diff_pay_med_bills_12m_10,SMKEV_A_2,SMKEV_A_10,heart_attack_2,heart_attack_10,cancer_2,cancer_10,ECIGEV_A_2,ECIGEV_A_10,prescription_meds_12m_2,prescription_meds_12m_10,high_blood_pressure_2.0,high_blood_pressure_10.0,chest_pain_2,chest_pain_10,last_visit_doctor_1,last_visit_doctor_2,last_visit_doctor_3,last_visit_doctor_4,last_visit_doctor_5,last_visit_doctor_6,last_visit_doctor_10,AFVET_A_2,AFVET_A_10,takes_value_of_1_if_SPOUSAGETC_A_value_is_NaN_or_in_tiny_category_1,income_from_ss/railroad_retirement_1.0,income_from_ss/railroad_retirement_2.0,income_from_ss/railroad_retirement_7.0,income_from_ss/railroad_retirement_8.0,income_from_ss/railroad_retirement_9.0,...,spouse education_8.0,spouse education_9.0,spouse education_10.0,spouse education_11.0,spouse education_97.0,spouse education_99.0,ERasthma_2.0,ERasthma_10.0,skipped_med_12m_1.0,skipped_med_12m_2.0,skipped_med_12m_10.0,CHOLLAST_A_1,CHOLLAST_A_2,CHOLLAST_A_3,CHOLLAST_A_4,CHOLLAST_A_5,CHOLLAST_A_6,CHOLLAST_A_10,no_health_ins_2,no_health_ins_10,anxiety_2,anxiety_10,asthma_2,asthma_10,have_health_ins_2,have_health_ins_10,high_cholesterol_2.0,high_cholesterol_10.0,alzheimers_2,alzheimers_10,asthma_attack12months_2.0,asthma_attack12months_10.0,usual_place_healthcare_2,usual_place_healthcare_10,depression_2,depression_10,visited_doctor_in_12m_1,pvt_health_ins_2,pvt_health_ins_10,target
0,-6.759123e-16,0.000000,-1.790015e-01,-0.605518,-0.548509,0,0,1,0,0,0,0,0,1,0,1,0,1,0,0,0,1,0,1,0,1,0,0,0,0,0,0,1,0,1,1,0,0,0,0,...,0,0,0,0,0,1,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,1,1,0,0
1,-1.012835e+00,0.000000,2.113454e-17,0.029519,-0.494282,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,1,0,1,0,0,0,1,0,1,0,1,0,0,0,1,0,1,0,0,0
2,-6.759123e-16,0.000000,2.113454e-17,-0.587878,1.240977,1,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,1,1,0,0,0,0,...,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,1,0,0,0,1,0,1,0,1,0,0,0,1,0,1,1,0,0
3,-6.759123e-16,-0.475159,-9.571763e-02,0.152999,0.264894,1,0,1,0,1,0,0,0,1,0,1,0,1,0,0,0,1,0,1,0,1,0,0,0,0,0,0,1,0,1,0,1,0,0,0,...,0,0,0,0,0,1,1,0,0,1,0,1,0,0,0,0,0,0,1,0,1,0,1,0,0,0,1,0,1,0,1,0,0,0,1,0,1,0,0,0
4,-2.249476e+00,0.000000,2.113454e-17,0.493149,-1.470366,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,1,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,1,0,1,0,0,0,1,0,1,0,1,0,0,0,1,0,1,0,0,0


In [9]:
train_data.shape

(25289, 145)

In [10]:
train = train_data.drop('target', axis = 1)
train_label = train_data[['target']]

In [11]:
test_data = pd.read_csv('FINAL_TEST.csv', header=0)
test = test_data.drop('target', axis=1)
test_label = test_data[['target']]

# Randomized Search with Cross Validation
- Decision Tree 
- Logistic Regression
- Random Forest Classifier
- Gradient Boosting Classifier

In [12]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV

skf = StratifiedKFold(n_splits=5)

In [20]:
from collections import defaultdict 
models = defaultdict(list)

### Decision Tree 

In [15]:
#Tuning Decision Tree by Randomized Grid Search

#Hyperparameter grid 
param_grid_dt = {
    'max_depth': [None] + list(np.linspace(3, 20).astype(int)), 
    'min_samples_split': [None] + list(np.linspace(50,2000).astype(int)), 
    'min_samples_leaf': [None] + list(np.linspace(10, 100).astype(int))
}

#Estimator 
dtree = DecisionTreeClassifier()

# Create the random search model
rs_dt = RandomizedSearchCV(dtree, param_grid_dt, n_jobs = -1, 
                        scoring = 'roc_auc', cv = skf, 
                        n_iter = 10, verbose = 1, random_state=50)

# Fit 
rs_dt.fit(train, train_label)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    6.7s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    7.1s finished


RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
                   error_score=nan,
                   estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0...
                                        'min_samples_leaf': [None, 10, 11,

In [19]:
best_dt = rs_dt.best_estimator_
print('CV score of best model: {}'.format(rs_dt.best_score_))

CV score of best model: 0.7511740072570281


In [25]:
#Test performance of best model
from sklearn.metrics import roc_auc_score
best_dt.fit(train, train_label)
pred = best_dt.predict_proba(test)
test_auc = roc_auc_score(test_label, pred[:,1])
print('Test score of best model: {}'.format(test_auc))

Test score of best model: 0.7592608445859824


In [26]:
models['Decision Tree RSCV'] = [rs_dt.best_score_, test_auc]

In [27]:
#Test performance of Default Decision Tree 
default_dt = DecisionTreeClassifier()
default_dt.fit(train, train_label)
pred = default_dt.predict_proba(test)
print(roc_auc_score(test_label, pred[:,1]))

0.6012546175561151


### Random Forest

In [28]:
#Tuning Random Forest by Randomized Grid Search

from sklearn.ensemble import RandomForestClassifier

# Hyperparameter grid
param_grid_rf = {
    'n_estimators': np.linspace(10, 200).astype(int),
    'max_depth': [None] + list(np.linspace(3, 20).astype(int)),
    'max_features': ['auto', 'sqrt', None] + list(np.arange(0.5, 1, 0.1)),
    'max_leaf_nodes': [None] + list(np.linspace(10, 50, 500).astype(int)),
    'min_samples_split': [2, 5, 10],
    'bootstrap': [True],
    'oob_score': [True]
}

# Estimator for use in random search
rf = RandomForestClassifier(random_state = 50)

# Create the random search model
rs_rf = RandomizedSearchCV(rf, param_grid_rf, n_jobs = -1, 
                        scoring = 'roc_auc', cv = skf, 
                        n_iter = 10, verbose = 1, random_state=50)

# Fit 
rs_rf.fit(train, train_label)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  2.8min finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:739: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
                   error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_sp...
                     

In [29]:
best_rf = rs_rf.best_estimator_
print('CV score of best model: {}'.format(rs_rf.best_score_))

CV score of best model: 0.7611600704780507


In [30]:
#Test performance of best model
best_rf.fit(train, train_label)
pred = best_rf.predict_proba(test)
test_auc = roc_auc_score(test_label, pred[:,1])
print('Test score of best model: {}'.format(test_auc))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


Test score of best model: 0.7689944336008097


In [31]:
models['Random Forest RSCV'] = [rs_rf.best_score_, test_auc]

### Gradient Boosting

In [33]:
#Tuning Gradient Boosting Classifier 

from sklearn.ensemble import GradientBoostingClassifier

# Hyperparameter grid
param_grid_gb = {
    'n_estimators': [1, 2, 4, 8, 16, 32, 64, 100, 200],
    'max_depth': np.linspace(1, 40, 40, endpoint=True), 
    'min_samples_split': np.linspace(0.1, 1.0, 10, endpoint=True), 
    'min_samples_leaf': np.linspace(0.1, 0.5, 5, endpoint=True), 
    'max_features': list(range(1,train.shape[1])), 
    'learning_rate': [1, 0.5, 0.25, 0.1, 0.05, 0.01]
}

# Estimator for use in random search
gb = GradientBoostingClassifier(random_state = 50)

# Create the random search model
rs_gb = RandomizedSearchCV(gb, param_grid_gb, n_jobs = -1, 
                        scoring = 'roc_auc', cv = skf, 
                        n_iter = 10, verbose = 1, random_state=50)

# Fit 
rs_gb.fit(train, train_label)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   12.5s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   13.5s finished
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
                   error_score=nan,
                   estimator=GradientBoostingClassifier(ccp_alpha=0.0,
                                                        criterion='friedman_mse',
                                                        init=None,
                                                        learning_rate=0.1,
                                                        loss='deviance',
                                                        max_depth=3,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                  

In [34]:
best_gb = rs_gb.best_estimator_
print('CV score of best model: {}'.format(rs_gb.best_score_))

CV score of best model: 0.7564307065844849


In [35]:
#Test performance of best model
best_gb.fit(train, train_label)
pred = best_gb.predict_proba(test)
test_auc = roc_auc_score(test_label, pred[:,1])
print('Test score of best model: {}'.format(test_auc))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Test score of best model: 0.7600869333099438


In [37]:
models['Gradient Boosting RSCV'] = [rs_gb.best_score_, test_auc]

### Logistic Regression

In [40]:
#Tuning Logistic Regression 

from sklearn.linear_model import LogisticRegression

C = np.logspace(-3, 3, num=10)
penalty = ['l1', 'l2']
solver = ['liblinear', 'saga']

param_grid_lr = dict(C=C, penalty=penalty, solver=solver)

lr = LogisticRegression(random_state = 50)

rs_lr = RandomizedSearchCV(lr, param_grid_lr, n_jobs = -1, 
                        scoring = 'roc_auc', cv = skf, 
                        n_iter = 10, verbose = 1, random_state=50)

# Fit 
rs_lr.fit(train, train_label)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  2.1min finished
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
                   error_score=nan,
                   estimator=LogisticRegression(C=1.0, class_weight=None,
                                                dual=False, fit_intercept=True,
                                                intercept_scaling=1,
                                                l1_ratio=None, max_iter=100,
                                                multi_class='auto', n_jobs=None,
                                                penalty='l2', random_state=50,
                                                solver='lbfgs', tol=0.0001,
                                                verbose=0, warm_start=False),
                   iid='dep..._jobs=-1,
                   param_distributions={'C': array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03]

In [41]:
best_lr = rs_lr.best_estimator_
print('CV score of best model: {}'.format(rs_lr.best_score_))

CV score of best model: 0.7643041547621804


In [42]:
#Test performance of best model
best_lr.fit(train, train_label)
pred = best_lr.predict_proba(test)
test_auc = roc_auc_score(test_label, pred[:,1])
print('Test score of best model: {}'.format(test_auc))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Test score of best model: 0.7707817810114438


In [43]:
models['Logistic Regression RSCV'] = [rs_lr.best_score_, test_auc]

In [45]:
models = dict(models)
pd.DataFrame.from_dict(models, orient = 'index', columns= ['CV_auc', 'Test_auc'])

,CV_auc,Test_auc
Decision Tree RSCV,0.751174,0.759261
Random Forest RSCV,0.761160,0.768994
Gradient Boosting RSCV,0.756431,0.760087
Logistic Regression RSCV,0.764304,0.770782


- Best = Logistic Regression 
- Next Best = Random Forest 

In [46]:
best_rf

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=12,
                       max_features=0.7999999999999999, max_leaf_nodes=33,
                       max_samples=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=188, n_jobs=None, oob_score=True,
                       random_state=50, verbose=0, warm_start=False)

In [47]:
best_lr

LogisticRegression(C=0.46415888336127775, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l1',
                   random_state=50, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [48]:
best_dt

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=16, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=96, min_samples_split=686,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [49]:
best_gb

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.25, loss='deviance', max_depth=22.0,
                           max_features=71, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=0.2, min_samples_split=0.2,
                           min_weight_fraction_leaf=0.0, n_estimators=64,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=50, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)